**IMT2200 - Introducción a Ciencia de Datos**  
**Pontificia Universidad Católica de Chile**  
**Instituto de Ingeniería Matemática y Computacional**
**2025 - Segundo Semestre**

**Profesor:** Rodrigo A. Carrasco 

---

# 🩺 **Enfermedades respiratorias en urgencias: pre y post COVID-19**

---

###  **Integrantes**
- Santiago González  
- Fernanda Le Roy  
- Joab Vivanco  

---

📅 **Fecha de entrega inicial:** 21 de octubre de 2025  


---


## Contexto y motivación

La pandemia de **COVID-19** produjo un cambio radical en los patrones de consulta por enfermedades respiratorias en los servicios de urgencia.  
Antes del 2020, la frecuencia con la que se presentaban virus respiratorios como **influenza**, **virus sincicial** y **adenovirus** determinaba los flujos de pacientes.  

Sin embargo, tras las medidas de confinamiento, el uso de mascarillas y el cambio en conductas sociales, se observaron variaciones en la **incidencia y distribución** de estas patologías.

La motivación de este proyecto es analizar si, efectivamente, la pandemia **alteró la cantidad de atenciones por enfermedades respiratorias en urgencias** y si estas modificaciones se han mantenido en los años posteriores al confinamiento.  

El problema es relevante porque los servicios de urgencia deben **planificar sus recursos humanos y materiales** de acuerdo con la demanda, y comprender estas tendencias puede facilitar la preparación frente a **futuras crisis sanitarias**.

La **audiencia objetivo** de este análisis son principalmente las **autoridades de salud pública** (Ministerio de Salud, Servicios de Salud regionales) y los **equipos de gestión hospitalaria**, quienes requieren información basada en datos para optimizar políticas de prevención, campañas de vacunación y asignación de recursos.


## Objetivos y preguntas de investigación

### Objetivo general

Analizar la **evolución de las consultas por enfermedades respiratorias en servicios de urgencia en Chile**, comparando el periodo previo y posterior a la pandemia de COVID-19, para identificar **tendencias y cambios significativos**.

---

### Objetivos específicos

- Describir la **incidencia de patologías respiratorias** atendidas en urgencias en la última década.  
- Comparar la **distribución temporal** de estas consultas antes y después de la pandemia.  
- Identificar **diferencias en prevalencia** según grupo etario, región y nivel de gravedad.  
- Explorar qué patologías han experimentado **mayores variaciones en frecuencia** tras el COVID-19.  
- Generar evidencia que pueda ser utilizada para la **planificación de servicios de salud**.

---

### Pregunta principal

> **¿Se ha observado un aumento de las enfermedades respiratorias que motivan consultas en urgencias tras la pandemia, en comparación con el periodo previo al COVID-19?**

---

### Preguntas secundarias

1. ¿Cómo influye el **tipo de patología** en la clasificación del nivel de urgencia (leve, moderado, grave)?  
2. ¿Existen **regiones o comunas** con una mayor incidencia de enfermedades respiratorias?  
3. ¿Qué **patologías respiratorias** han sido más prevalentes en la última década y cómo ha cambiado su distribución a lo largo de los años?  
4. ¿Cuál es la **relación entre los rangos etarios** y la prevalencia de enfermedades respiratorias, y qué patologías son más comunes en cada grupo de edad?  

---

📌 En conjunto, estas preguntas abordan las dimensiones **clínica, geográfica, temporal y poblacional** del fenómeno estudiado, aportando una visión integral que permitirá responder la pregunta de investigación principal.


---

🧭 En las siguientes secciones se presentan los datos utilizados, el proceso de limpieza y un análisis exploratorio que busca responder las preguntas planteadas.

## Descripción de los datos

Los datos utilizados provienen de la **Plataforma de Datos Abiertos del Gobierno de Chile**, específicamente del conjunto **“Atenciones de urgencia por causas respiratorias”**  
(https://datos.gob.cl/dataset/atenciones-de-urgencia-causas-respiratorias).

El registro se realiza **semanalmente** e incluye información sobre las atenciones por enfermedades respiratorias en establecimientos de salud de todo el país, principalmente del **sector público**.

Cada fila del dataset representa un **establecimiento en una semana específica**, e incluye tanto variables **administrativas** como **epidemiológicas**.

---

###  Variables principales

- **EstablecimientoCodigo:** Identificador único del establecimiento.  
- **RegionGlosa / ComunaGlosa:** Región y comuna donde se ubica el establecimiento.  
- **Causa:** Tipo de enfermedad respiratoria (ej. influenza, virus sincicial, adenovirus).  
- **Anio / SemanaEstadistica:** Año y semana epidemiológica del registro.  
- **NumTotal:** Número total de consultas por causas respiratorias.  
- **NumMenor1Anio, Num1a4Anios, Num5a14Anios, Num15a64Anios, Num65Mas:** Distribución de consultas por grupo etario.  
- **NivelAtencion / TipoEstablecimiento / NivelComplejidad:** Clasificación administrativa y operativa del centro de salud.

---

 En términos generales, el dataset contiene **múltiples años (2013 en adelante)** y permite analizar la **evolución temporal, distribución geográfica y diferencias por edad o tipo de patología** en las consultas de urgencia respiratoria en Chile.

Antes de realizar el análisis exploratorio, es necesario definir la estrategia de procesamiento que seguiremos con los datos.

###  Estrategia de procesamiento: ELT vs ETL

Existen dos enfoques comunes para trabajar con datos:

- **ETL (Extract → Transform → Load):**  
 
- **ELT (Extract → Load → Transform):**  
 

### En este proyecto utilizaremos ELT

Optamos por **ELT** porque:

- El dataset es **público, estructurado y grande**, con posibles ajustes menores (nulos, formato de texto, nombres).  
- Nos permite **mantener una copia cruda** en la carpeta `/data/raw/` y trabajar las transformaciones directamente en el notebook.  
- Facilita la **reproducibilidad y trazabilidad** del proceso, mostrando cada paso de limpieza dentro del análisis.

---

##  Limpieza y preparación de los datos

In [2]:
# librerías usadas
import pandas as pd
import numpy as np
import matplotlib as plt
import pyarrow.parquet as pq

In [4]:
data = pq.read_table("../data/atenciones_respiratorias.parquet")

data = data.to_pandas()
data

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,Anio,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas
0,105805,SAR Tierras Blancas,4,Región De Coquimbo,4102,Coquimbo,5,Servicio de Salud Coquimbo,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,2015,29,4,IRA Alta (J00-J06),216,13,24,43,109,27
1,116101,Hospital de Teno,7,Región Del Maule,7308,Teno,16,Servicio de Salud Del Maule,Hospital,Servicio de Salud,...,2016,30,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,3,0,0,0,2,1
2,200759,SUR Morza,7,Región Del Maule,7308,Teno,16,Servicio de Salud Del Maule,Servicio de Urgencia Rural (SUR),Municipal,...,2024,3,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0
3,109805,SAR La Pincoya,13,Región Metropolitana de Santiago,13107,Huechuraba,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,2025,36,4,IRA Alta (J00-J06),180,4,30,38,99,9
4,109809,SAR Conchalí,13,Región Metropolitana de Santiago,13104,Conchalí,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,2020,24,8,Crisis obstructiva bronquial (J40-J46),0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3267875,122100,Hospital Clínico Regional (Valdivia),14,Región De Los Ríos,14101,Valdivia,22,Servicio de Salud Los Rios,Hospital,Servicio de Salud,...,2014,14,8,Crisis obstructiva bronquial (J40-J46),81,8,23,9,25,16
3267876,200753,SUR Cariquima,1,Región De Tarapacá,1403,Colchane,2,Servicio de Salud Tarapacá,Servicio de Urgencia Rural (SUR),Municipal,...,2024,9,34,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",0,0,0,0,0,0
3267877,109802,SAPU Lucas Sierra,13,Región Metropolitana de Santiago,13104,Conchalí,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia (SAPU),Municipal,...,2022,39,5,Influenza (J09-J11),6,0,0,3,3,0
3267878,124810,SAR Puerto Varas,10,Región De Los Lagos,10109,Puerto Varas,24,Servicio de Salud Del Reloncaví,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,2022,50,7,Bronquitis/bronquiolitis aguda (J20-J21),61,6,16,16,15,8


In [6]:
# Identificar columnas con valores faltantes 
data.isnull().sum()

EstablecimientoCodigo        17714
EstablecimientoGlosa         17714
RegionCodigo                 17714
RegionGlosa                  17714
ComunaCodigo                 17714
ComunaGlosa                  17714
ServicioSaludCodigo          17714
ServicioSaludGlosa           17714
TipoEstablecimiento          17714
DependenciaAdministrativa    17714
NivelAtencion                17714
TipoUrgencia                 17714
Latitud                      17714
Longitud                     17714
NivelComplejidad             17714
Anio                             0
SemanaEstadistica                0
OrdenCausa                       0
Causa                            0
NumTotal                         0
NumMenor1Anio                    0
Num1a4Anios                      0
Num5a14Anios                     0
Num15a64Anios                    0
Num65oMas                        0
dtype: int64

se realiza una primera exploración de los valores faltantes en el dataset mediante isnull().sum()

Luego de identificar las columnas con valores faltantes, se calcula el porcentaje de nulos en cada variable (isnull().mean())

In [105]:
# #PROPUESTA JOAB
# df_nulos = data[data.isnull().any(axis=1)]
# df_nulos

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,Anio,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas
7,None,None,None,None,None,None,None,None,None,None,...,2022,5,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0
219,None,None,None,None,None,None,None,None,None,None,...,2025,40,4,IRA Alta (J00-J06),11,0,2,1,8,0
416,None,None,None,None,None,None,None,None,None,None,...,2025,38,6,Neumonía (J12-J18),1,0,0,0,1,0
521,None,None,None,None,None,None,None,None,None,None,...,2024,3,7,Bronquitis/bronquiolitis aguda (J20-J21),2,0,0,1,1,0
674,None,None,None,None,None,None,None,None,None,None,...,2024,52,10,"Covid-19, Virus no identificado U07.2",1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3267299,None,None,None,None,None,None,None,None,None,None,...,2024,34,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,52,3,5,16,23,5
3267354,None,None,None,None,None,None,None,None,None,None,...,2020,33,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,2,0,0,1,0,1
3267526,None,None,None,None,None,None,None,None,None,None,...,2024,41,6,Neumonía (J12-J18),0,0,0,0,0,0
3267583,None,None,None,None,None,None,None,None,None,None,...,2020,20,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0


In [10]:
# verificamos ahora si estas columnas con datos null son relevantes
# para evaluar de que forma borrarlas o ajustar los datos de forma pertinente
data.isnull().mean().sort_values(ascending=False)

EstablecimientoCodigo        0.005421
TipoEstablecimiento          0.005421
NivelComplejidad             0.005421
Longitud                     0.005421
EstablecimientoGlosa         0.005421
TipoUrgencia                 0.005421
NivelAtencion                0.005421
DependenciaAdministrativa    0.005421
Latitud                      0.005421
ServicioSaludGlosa           0.005421
ServicioSaludCodigo          0.005421
ComunaGlosa                  0.005421
ComunaCodigo                 0.005421
RegionGlosa                  0.005421
RegionCodigo                 0.005421
Anio                         0.000000
SemanaEstadistica            0.000000
OrdenCausa                   0.000000
Causa                        0.000000
NumTotal                     0.000000
NumMenor1Anio                0.000000
Num1a4Anios                  0.000000
Num5a14Anios                 0.000000
Num15a64Anios                0.000000
Num65oMas                    0.000000
dtype: float64

Podemos interpretar esto como que las columnas que antes parecían llenas de valores nulos en realidad solo tienen alrededor del 0.5% de valores faltantes, por ende, los datos están bastante completos pero se veían como hartos valores faltantes porque el dataset es grande, pero quizá estos valores nulos no son triviales.

Si observamos pareciera ser que los valores nulos tienen que ver con valores que entregan información geográfica e institucional de los contagios, es pertinente analizarlos...

In [14]:
# NaN analisis por cada columna
data[data["EstablecimientoCodigo"].isna()].head(10)

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,Anio,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas
7,None,None,None,None,None,None,None,None,None,None,...,2022,5,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0
219,None,None,None,None,None,None,None,None,None,None,...,2025,40,4,IRA Alta (J00-J06),11,0,2,1,8,0
416,None,None,None,None,None,None,None,None,None,None,...,2025,38,6,Neumonía (J12-J18),1,0,0,0,1,0
521,None,None,None,None,None,None,None,None,None,None,...,2024,3,7,Bronquitis/bronquiolitis aguda (J20-J21),2,0,0,1,1,0
674,None,None,None,None,None,None,None,None,None,None,...,2024,52,10,"Covid-19, Virus no identificado U07.2",1,0,0,0,1,0
871,None,None,None,None,None,None,None,None,None,None,...,2024,31,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0
957,None,None,None,None,None,None,None,None,None,None,...,2020,53,9,"Otra causa respiratoria (J22, J30-J39, J47, J6...",1,0,0,1,0,0
1619,None,None,None,None,None,None,None,None,None,None,...,2024,29,7,Bronquitis/bronquiolitis aguda (J20-J21),71,0,13,16,29,13
2065,None,None,None,None,None,None,None,None,None,None,...,2024,14,6,Neumonía (J12-J18),3,0,1,0,2,0
2130,None,None,None,None,None,None,None,None,None,None,...,2018,25,9,"Otra causa respiratoria (J22, J30-J39, J47, J6...",14,0,1,6,7,0


In [16]:
columnas_nulas = [
    "EstablecimientoCodigo", "EstablecimientoGlosa", "RegionCodigo",
    "RegionGlosa", "ComunaCodigo", "ComunaGlosa", "ServicioSaludCodigo",
    "ServicioSaludGlosa", "TipoEstablecimiento", "DependenciaAdministrativa",
    "NivelAtencion", "TipoUrgencia", "Latitud", "Longitud", "NivelComplejidad"
]
data[columnas_nulas].isnull().all(axis=1).sum()

17714

Con esta información inicial se busca tener claridad sobre el contenido y la estructura del dataset antes de aplicar procesos de limpieza y transformación.

Luego de hacer esto nos damos cuenta que las filas sí tienen datos de año, semana, causa y número de casos, pero sin ningún contexto de dónde provienen (no sabemos de qué comuna, hospital, ni región). Seguramente son registros agregados o globales (por ejemplo, totales nacionales o semanales) que no se asignaron a un establecimiento o comuna específica. Como posteriormente vamos a evaluar el total de contagios, no eliminaremos los NaN, sino que dejaremos 2 dataset distintos, uno con los NaN y otro sin para cuando tengamos que graficar o analizar según institución. 

In [20]:
df_con_nan = data.copy()

# no es bueno en nuestro caso borrar filas completas mejor solo columnas vacías
df_sin_nan = data.dropna(subset=["EstablecimientoCodigo", "EstablecimientoGlosa", "RegionCodigo",
    "RegionGlosa", "ComunaCodigo", "ComunaGlosa", "ServicioSaludCodigo",
    "ServicioSaludGlosa", "TipoEstablecimiento", "DependenciaAdministrativa",
    "NivelAtencion", "TipoUrgencia", "Latitud", "Longitud", "NivelComplejidad"])


### PROPUESTA DE CAMBIOS JOAB

Tengo la idea que como nuestra propuesta inicial trata sobre PRE y POST pandemia, nos enfoquemos sólo en analizar 3 años antes y después de pandemia. Que sean 2017, 2018, 2019, 2023, 2024 y 2025. 

In [125]:
#Revisé si es que tenemos muchos datos como para trabajar con ellos, para no elegir años que tengan pocos datos y afecten el resultado.
df_con_nan[["Anio"]].value_counts()

Anio
2024    385452
2023    381912
2022    379620
2021    378000
2020    324682
2025    307980
2019    192522
2018    186912
2017    186032
2016    184832
2015    184480
2014    175456
Name: count, dtype: int64

In [127]:
#Ahora hago el filtro según los años que les dije recién
df_per_year = df_con_nan[(df_con_nan["Anio"] >= 2017) & (df_con_nan["Anio"] >= 2023)]
df_per_year.count()


EstablecimientoCodigo        1064484
EstablecimientoGlosa         1064484
RegionCodigo                 1064484
RegionGlosa                  1064484
ComunaCodigo                 1064484
ComunaGlosa                  1064484
ServicioSaludCodigo          1064484
ServicioSaludGlosa           1064484
TipoEstablecimiento          1064484
DependenciaAdministrativa    1064484
NivelAtencion                1064484
TipoUrgencia                 1064484
Latitud                            0
Longitud                           0
NivelComplejidad             1064484
Anio                         1075344
SemanaEstadistica            1075344
OrdenCausa                   1075344
Causa                        1075344
NumTotal                     1075344
NumMenor1Anio                1075344
Num1a4Anios                  1075344
Num5a14Anios                 1075344
Num15a64Anios                1075344
Num65oMas                    1075344
Fecha                        1075344
dtype: int64

In [129]:
df_per_year.isnull().sum()

EstablecimientoCodigo          10860
EstablecimientoGlosa           10860
RegionCodigo                   10860
RegionGlosa                    10860
ComunaCodigo                   10860
ComunaGlosa                    10860
ServicioSaludCodigo            10860
ServicioSaludGlosa             10860
TipoEstablecimiento            10860
DependenciaAdministrativa      10860
NivelAtencion                  10860
TipoUrgencia                   10860
Latitud                      1075344
Longitud                     1075344
NivelComplejidad               10860
Anio                               0
SemanaEstadistica                  0
OrdenCausa                         0
Causa                              0
NumTotal                           0
NumMenor1Anio                      0
Num1a4Anios                        0
Num5a14Anios                       0
Num15a64Anios                      0
Num65oMas                          0
Fecha                              0
dtype: int64

In [115]:
df_per_year.columns

Index(['EstablecimientoCodigo', 'EstablecimientoGlosa', 'RegionCodigo',
       'RegionGlosa', 'ComunaCodigo', 'ComunaGlosa', 'ServicioSaludCodigo',
       'ServicioSaludGlosa', 'TipoEstablecimiento',
       'DependenciaAdministrativa', 'NivelAtencion', 'TipoUrgencia', 'Latitud',
       'Longitud', 'NivelComplejidad', 'Anio', 'SemanaEstadistica',
       'OrdenCausa', 'Causa', 'NumTotal', 'NumMenor1Anio', 'Num1a4Anios',
       'Num5a14Anios', 'Num15a64Anios', 'Num65oMas', 'Fecha'],
      dtype='object')

In [121]:
dfff = df_per_year[['EstablecimientoCodigo', 'EstablecimientoGlosa', 'RegionCodigo',
       'RegionGlosa', 'ComunaCodigo', 'ComunaGlosa', 'ServicioSaludCodigo',
       'ServicioSaludGlosa', 'TipoEstablecimiento',
       'DependenciaAdministrativa', 'NivelAtencion', 'TipoUrgencia', 'NivelComplejidad', 'Anio', 'SemanaEstadistica',
       'OrdenCausa', 'Causa', 'NumTotal', 'NumMenor1Anio', 'Num1a4Anios',
       'Num5a14Anios', 'Num15a64Anios', 'Num65oMas', 'Fecha']]

In [123]:
df_nulos = dfff[dfff.isnull().any(axis=1)]
df_nulos

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
219,None,None,None,None,None,None,None,None,None,None,...,40,4,ira alta (j00-j06),11,0,2,1,8,0,2025-10-06
416,None,None,None,None,None,None,None,None,None,None,...,38,6,neumonía (j12-j18),1,0,0,0,1,0,2025-09-22
521,None,None,None,None,None,None,None,None,None,None,...,3,7,bronquitis/bronquiolitis aguda (j20-j21),2,0,0,1,1,0,2024-01-22
674,None,None,None,None,None,None,None,None,None,None,...,52,10,"covid-19, virus no identificado u07.2",1,0,0,0,1,0,2024-12-30
871,None,None,None,None,None,None,None,None,None,None,...,31,32,hospitalizaciones por causas sistema respiratorio,0,0,0,0,0,0,2024-08-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266948,None,None,None,None,None,None,None,None,None,None,...,49,32,hospitalizaciones por causas sistema respiratorio,0,0,0,0,0,0,2024-12-09
3267217,None,None,None,None,None,None,None,None,None,None,...,11,11,"covid-19, virus identificado u07.1",0,0,0,0,0,0,2025-03-17
3267299,None,None,None,None,None,None,None,None,None,None,...,34,3,total causas sistema respiratorio,52,3,5,16,23,5,2024-08-26
3267526,None,None,None,None,None,None,None,None,None,None,...,41,6,neumonía (j12-j18),0,0,0,0,0,0,2024-10-14


In [137]:
df_per_year[["Latitud", "Longitud"]].isnull().sum()

Latitud     1075344
Longitud    1075344
dtype: int64

In [143]:
df_per_year['Latitud'] = pd.to_numeric(df_per_year['Latitud'], errors='coerce')
df_per_year['Longitud'] = pd.to_numeric(df_per_year['Longitud'], errors='coerce')
df_per_year[["Latitud", "Longitud"]].isnull().sum()

C:\Users\joabv\AppData\Local\Temp\ipykernel_9420\2968209164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_per_year['Latitud'] = pd.to_numeric(df_per_year['Latitud'], errors='coerce')
C:\Users\joabv\AppData\Local\Temp\ipykernel_9420\2968209164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_per_year['Longitud'] = pd.to_numeric(df_per_year['Longitud'], errors='coerce')


Latitud     1075344
Longitud    1075344
dtype: int64

# Dejamos entonces los datos parcelados desde ahora!

Desde aquí en adelante comienza la etapa de limpieza del dataset. El objetivo es revisar la estructura general de los datos, detectar valores nulos o inconsistentes, y preparar las variables para el análisis exploratorio posterior.

In [24]:
# tipo de cada columna
print(df_con_nan.dtypes)
print(df_sin_nan.dtypes)

EstablecimientoCodigo        object
EstablecimientoGlosa         object
RegionCodigo                 object
RegionGlosa                  object
ComunaCodigo                 object
ComunaGlosa                  object
ServicioSaludCodigo          object
ServicioSaludGlosa           object
TipoEstablecimiento          object
DependenciaAdministrativa    object
NivelAtencion                object
TipoUrgencia                 object
Latitud                      object
Longitud                     object
NivelComplejidad             object
Anio                          int32
SemanaEstadistica             int32
OrdenCausa                    int32
Causa                        object
NumTotal                      int32
NumMenor1Anio                 int32
Num1a4Anios                   int32
Num5a14Anios                  int32
Num15a64Anios                 int32
Num65oMas                     int32
dtype: object
EstablecimientoCodigo        object
EstablecimientoGlosa         object
RegionCodigo  

En este paso se inspeccionan los tipos de datos de ambas versiones del DataFrame (df_con_nan y df_sin_nan) con esto podemos confirmar si las columnas están correctamente tipadas

In [27]:
# cambiamos latitud y longitud
df_con_nan = df_con_nan.copy()
df_sin_nan = df_sin_nan.copy()

df_con_nan['Latitud'] = pd.to_numeric(df_con_nan['Latitud'], errors='coerce')
df_con_nan['Longitud'] = pd.to_numeric(df_con_nan['Longitud'], errors='coerce')
df_sin_nan['Latitud'] = pd.to_numeric(df_sin_nan['Latitud'], errors='coerce')
df_sin_nan['Longitud'] = pd.to_numeric(df_sin_nan['Longitud'], errors='coerce')

Cambiamos latitud y longitud para ambos ya que nos percatamos que la idea es que sean numéricos para poder graficar posteriormente sin problema

In [30]:
# creamos la columna fecha para ambas
df_con_nan['Fecha'] = pd.to_datetime(
    df_con_nan['Anio'].astype(str) + df_con_nan['SemanaEstadistica'].astype(str) + '1',
    format='%Y%U%w', errors='coerce'
)

df_sin_nan['Fecha'] = pd.to_datetime(
    df_sin_nan['Anio'].astype(str) + df_sin_nan['SemanaEstadistica'].astype(str) + '1',
    format='%Y%U%w', errors='coerce'
)

Además de eso, agregamos la columna fecha que nos permitirá graficar y hacer un análisis más en detalle por año, mes, entre otras cosas.

In [33]:
# verificamos si la columna se creó bien o algunas fechas quedaron conNat 
fechas_invalidas = df_con_nan[df_con_nan["Fecha"].isna()][["Anio", "SemanaEstadistica"]]
print("Fechas inválidas:", len(fechas_invalidas))
fechas_invalidas.head()

Fechas inválidas: 0


,Anio,SemanaEstadistica


In [35]:
df_con_nan.head()

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
0,105805,SAR Tierras Blancas,4,Región De Coquimbo,4102,Coquimbo,5,Servicio de Salud Coquimbo,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,29,4,IRA Alta (J00-J06),216,13,24,43,109,27,2015-07-20
1,116101,Hospital de Teno,7,Región Del Maule,7308,Teno,16,Servicio de Salud Del Maule,Hospital,Servicio de Salud,...,30,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,3,0,0,0,2,1,2016-07-25
2,200759,SUR Morza,7,Región Del Maule,7308,Teno,16,Servicio de Salud Del Maule,Servicio de Urgencia Rural (SUR),Municipal,...,3,32,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,0,0,0,0,0,0,2024-01-22
3,109805,SAR La Pincoya,13,Región Metropolitana de Santiago,13107,Huechuraba,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,36,4,IRA Alta (J00-J06),180,4,30,38,99,9,2025-09-08
4,109809,SAR Conchalí,13,Región Metropolitana de Santiago,13104,Conchalí,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,24,8,Crisis obstructiva bronquial (J40-J46),0,0,0,0,0,0,2020-06-15


Ahora revisamos los duplicados.


In [38]:
# Ver cuántas filas duplicadas hay
duplicados = df_con_nan.duplicated().sum()
print("Filas duplicadas:", duplicados)

Filas duplicadas: 5588


In [40]:
dup_por_semana = df_con_nan.duplicated(subset=['Anio', 'SemanaEstadistica', 'Causa'])
print("Duplicados por año-semana-causa:", dup_por_semana.sum())

# visualizamos
df_con_nan[dup_por_semana].head()

Duplicados por año-semana-causa: 3261576


,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
154,116101,Hospital de Teno,7,Región Del Maule,7308,Teno,16,Servicio de Salud Del Maule,Hospital,Servicio de Salud,...,24,4,IRA Alta (J00-J06),62,7,15,11,27,2,2015-06-15
171,112815,SAPU San Luis,13,Región Metropolitana de Santiago,13122,Peñalolén,12,Servicio de Salud Metropolitano Oriente,Servicio de Atención Primaria de Urgencia (SAPU),Municipal,...,11,8,Crisis obstructiva bronquial (J40-J46),6,0,5,1,0,0,2019-03-18
220,200130,SAPU Peñaflor,13,Región Metropolitana de Santiago,13605,Peñaflor,10,Servicio de Salud Metropolitano Occidente,Servicio de Atención Primaria de Urgencia (SAPU),Municipal,...,8,34,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",0,0,0,0,0,0,2024-02-26
221,200853,SUR Placilla,6,Región Del Libertador Gral. B. O'Higgins,6308,Placilla,15,Servicio de Salud Del Libertador B.O'Higgins,Servicio de Urgencia Rural (SUR),Municipal,...,23,3,TOTAL CAUSAS SISTEMA RESPIRATORIO,56,3,12,17,22,2,2022-06-06
248,116832,SAR Constitución,7,Región Del Maule,7102,Constitución,16,Servicio de Salud Del Maule,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,35,9,"Otra causa respiratoria (J22, J30-J39, J47, J6...",4,1,0,1,2,0,2022-08-29


### Se visualizan las primeras filas con posibles duplicados para revisarlos.
Este paso permite determinar si los duplicados son efectivamente errores (registros repetidos) o si corresponden a diferentes establecimientos dentro de la misma semana y causa, en cuyo caso no deberían eliminarse.

In [43]:
# vemos si eliminamos o no
dup_detallado = df_con_nan.duplicated(
    subset=['EstablecimientoCodigo', 'Anio', 'SemanaEstadistica', 'Causa', 'NumTotal']
)
print("Duplicados verdaderos:", dup_detallado.sum())
df_con_nan[dup_detallado].head()

Duplicados verdaderos: 6174


,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
20994,None,None,None,None,None,None,None,None,None,None,...,28,34,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",0,0,0,0,0,0,2024-07-15
56792,None,None,None,None,None,None,None,None,None,None,...,28,33,"HOSPITALIZACIONES COVID-19, VIRUS NO IDENTIFIC...",0,0,0,0,0,0,2025-07-14
74797,None,None,None,None,None,None,None,None,None,None,...,28,11,"Covid-19, Virus identificado U07.1",0,0,0,0,0,0,2025-07-14
76295,None,None,None,None,None,None,None,None,None,None,...,21,11,"Covid-19, Virus identificado U07.1",0,0,0,0,0,0,2025-05-26
88403,None,None,None,None,None,None,None,None,None,None,...,46,10,"Covid-19, Virus no identificado U07.2",0,0,0,0,0,0,2024-11-18


In [45]:
# vemos si eliminamos o no
dup_detallado = df_sin_nan.duplicated(
    subset=['EstablecimientoCodigo', 'Anio', 'SemanaEstadistica', 'Causa', 'NumTotal']
)
print("Duplicados verdaderos:", dup_detallado.sum())
df_sin_nan[dup_detallado].head()

Duplicados verdaderos: 0


,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha


## Análisis de duplicados y decisión sobre los datasets

Al revisar los posibles duplicados en el dataset con valores nulos (`df_con_nan`), se observó que varias filas aparecen repetidas porque carecen de información geográfica e institucional (columnas como **Establecimiento**, **Región** o **Comuna** aparecen como `None`).  
Estas filas representan **registros agregados o globales**, por ejemplo, **totales nacionales o semanales** que no se asignaron a un establecimiento específico.

En cambio, al realizar la misma verificación en el dataset sin valores nulos (`df_sin_nan`), **no se encontraron duplicados verdaderos** (`Duplicados verdaderos: 0`), lo que confirma que la base de datos limpia mantiene **registros únicos por establecimiento, año, semana y causa**.

---

### Decisión de limpieza

Dado que las filas con valores nulos podrían aportar información útil para análisis globales (por ejemplo, el **total de casos semanales a nivel nacional**), se decidió mantener **dos versiones del dataset**:

- **`df_con_nan`** → incluye todos los registros, incluso los agregados sin identificación institucional.  
- **`df_sin_nan`** → conserva únicamente las filas con información completa, útil para análisis por **establecimiento o región**.

Esta decisión permitirá usar uno u otro dataset según el tipo de análisis que se realice (por ejemplo, **comparaciones regionales** vs. **tendencias generales**).


In [48]:
## Exploración de las causas
# revisamos qué tipos de enfermedades o condiciones respiratorias hay en el dataset
causas_con_nan = df_con_nan.drop_duplicates(subset=["Causa"])
lista_causas_con_nan = causas_con_nan["Causa"].tolist()

print("Lista de causas (df_con_nan):")
print(lista_causas_con_nan)

Lista de causas (df_con_nan):
['IRA Alta (J00-J06)', 'HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO', 'Crisis obstructiva bronquial (J40-J46)', 'Otra causa respiratoria (J22, J30-J39, J47, J60-J98)', 'Neumonía (J12-J18)', 'Covid-19, Virus no identificado U07.2', 'Bronquitis/bronquiolitis aguda (J20-J21)', 'HOSPITALIZACIONES COVID-19, VIRUS NO IDENTIFICADO U07.2', 'TOTAL CAUSAS SISTEMA RESPIRATORIO', 'HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO U07.1', 'Influenza (J09-J11)', 'Covid-19, Virus identificado U07.1']


In [50]:
# Al observar la lista, notamos algunas diferencias en la escritura,
# por ejemplo: "Covid-19, Virus identificado U07.1" vs "COVID-19, Virus identificado U07.1".
# entonces para mantener consistencia, convertimos todos los nombres a minúsculas y eliminamos espacios extra.

df_con_nan["Causa"] = df_con_nan["Causa"].str.strip().str.lower()
df_sin_nan["Causa"] = df_sin_nan["Causa"].str.strip().str.lower()

# Revisamos nuevamente la lista de causas para confirmar los cambios
lista_causas_con_nan = df_con_nan["Causa"].drop_duplicates().tolist()
print("Lista de causas homogeneizada:")
print(lista_causas_con_nan)

Lista de causas homogeneizada:
['ira alta (j00-j06)', 'hospitalizaciones por causas sistema respiratorio', 'crisis obstructiva bronquial (j40-j46)', 'otra causa respiratoria (j22, j30-j39, j47, j60-j98)', 'neumonía (j12-j18)', 'covid-19, virus no identificado u07.2', 'bronquitis/bronquiolitis aguda (j20-j21)', 'hospitalizaciones covid-19, virus no identificado u07.2', 'total causas sistema respiratorio', 'hospitalizaciones covid-19, virus identificado u07.1', 'influenza (j09-j11)', 'covid-19, virus identificado u07.1']


In [52]:
# Creamos un subconjunto por cada causa del dataset (df_con_nan)
causa_0 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[0]]  #"IRA Alta (j00-j06)"
causa_1 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[1]]  # muy general "Hospitalizaciones por causas sistema respiratorio"
causa_2 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[2]] # "Crisis obstructuva bronquial (j40-j46)"
causa_3 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[3]] #"Otra causa respiratoria (j22,j30-j39,j47,j60-j98)"
causa_4 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[4]] #"Neumonia (j12-j18)"
causa_5 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[5]] #"Covid-19, Virus no identificado U07.2"
causa_6 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[6]] #"Bronquitis/bronquiolitis aguda (j20-j21)"
causa_7 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[7]] #"HOSPITALIZACIONES COVID-19, VIRUS NO IDENTIFICADO U07.2"
causa_8 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[8]] # "TOTAL CAUSAS SISTEMA RESPIRATORIO"
causa_9 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[9]] # "HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO U07.1"
causa_10 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[10]] # "Influenza (j09-j11)"
causa_11 = df_con_nan[df_con_nan["Causa"] == lista_causas_con_nan[11]] #"Covid-19, virus identificado U07.1"

# visualizamos para comprobar
causa_0.head()

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
0,105805,SAR Tierras Blancas,4,Región De Coquimbo,4102,Coquimbo,5,Servicio de Salud Coquimbo,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,29,4,ira alta (j00-j06),216,13,24,43,109,27,2015-07-20
3,109805,SAR La Pincoya,13,Región Metropolitana de Santiago,13107,Huechuraba,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,36,4,ira alta (j00-j06),180,4,30,38,99,9,2025-09-08
8,109824,SAPU Presidente Salvador Allende Gossens,13,Región Metropolitana de Santiago,13125,Quilicura,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia (SAPU),Municipal,...,28,4,ira alta (j00-j06),19,1,4,1,13,0,2020-07-13
12,115114,Hospital de Litueche,6,Región Del Libertador Gral. B. O'Higgins,6203,Litueche,15,Servicio de Salud Del Libertador B.O'Higgins,Hospital,Servicio de Salud,...,10,4,ira alta (j00-j06),12,0,2,3,7,0,2016-03-07
13,122101,Hospital de Corral,14,Región De Los Ríos,14102,Corral,22,Servicio de Salud Los Rios,Hospital,Servicio de Salud,...,33,4,ira alta (j00-j06),30,4,2,7,15,2,2018-08-20


In [53]:
# Creamos un subconjunto por cada causa del dataset (df_sin_nan)
causa_0 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[0]]  #"IRA Alta (j00-j06)"
causa_1 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[1]]  # muy general "Hospitalizaciones por causas sistema respiratorio"
causa_2 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[2]] # "Crisis obstructuva bronquial (j40-j46)"
causa_3 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[3]] #"Otra causa respiratoria (j22,j30-j39,j47,j60-j98)"
causa_4 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[4]] #"Neumonia (j12-j18)"
causa_5 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[5]] #"Covid-19, Virus no identificado U07.2"
causa_6 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[6]] #"Bronquitis/bronquiolitis aguda (j20-j21)"
causa_7 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[7]] #"HOSPITALIZACIONES COVID-19, VIRUS NO IDENTIFICADO U07.2"
causa_8 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[8]] # "TOTAL CAUSAS SISTEMA RESPIRATORIO"
causa_9 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[9]] # "HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO U07.1"
causa_10 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[10]] # "Influenza (j09-j11)"
causa_11 = df_sin_nan[df_sin_nan["Causa"] == lista_causas_con_nan[11]] #"Covid-19, virus identificado U07.1"

# visualizamos para comprobar
causa_0.head()

,EstablecimientoCodigo,EstablecimientoGlosa,RegionCodigo,RegionGlosa,ComunaCodigo,ComunaGlosa,ServicioSaludCodigo,ServicioSaludGlosa,TipoEstablecimiento,DependenciaAdministrativa,...,SemanaEstadistica,OrdenCausa,Causa,NumTotal,NumMenor1Anio,Num1a4Anios,Num5a14Anios,Num15a64Anios,Num65oMas,Fecha
0,105805,SAR Tierras Blancas,4,Región De Coquimbo,4102,Coquimbo,5,Servicio de Salud Coquimbo,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,29,4,ira alta (j00-j06),216,13,24,43,109,27,2015-07-20
3,109805,SAR La Pincoya,13,Región Metropolitana de Santiago,13107,Huechuraba,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia de A...,Municipal,...,36,4,ira alta (j00-j06),180,4,30,38,99,9,2025-09-08
8,109824,SAPU Presidente Salvador Allende Gossens,13,Región Metropolitana de Santiago,13125,Quilicura,9,Servicio de Salud Metropolitano Norte,Servicio de Atención Primaria de Urgencia (SAPU),Municipal,...,28,4,ira alta (j00-j06),19,1,4,1,13,0,2020-07-13
12,115114,Hospital de Litueche,6,Región Del Libertador Gral. B. O'Higgins,6203,Litueche,15,Servicio de Salud Del Libertador B.O'Higgins,Hospital,Servicio de Salud,...,10,4,ira alta (j00-j06),12,0,2,3,7,0,2016-03-07
13,122101,Hospital de Corral,14,Región De Los Ríos,14102,Corral,22,Servicio de Salud Los Rios,Hospital,Servicio de Salud,...,33,4,ira alta (j00-j06),30,4,2,7,15,2,2018-08-20


In [54]:
## Clasificación de causas en dos categorías generales

# Identificamos todas las causas que contienen la palabra "hospital" o "hospitaliz"
# como hospitalizaciones, y las demás como contagios ENTONCES SEPARAMOS.

mask_hosp_con = df_con_nan["Causa"].str.contains("hospital", case=False, na=False)
mask_hosp_sin = df_sin_nan["Causa"].str.contains("hospital", case=False, na=False)

df_hospitalizaciones_con = df_con_nan[mask_hosp_con]
df_contagios_con = df_con_nan[~mask_hosp_con]

df_hospitalizaciones_sin = df_sin_nan[mask_hosp_sin]
df_contagios_sin = df_sin_nan[~mask_hosp_sin]

print("Hospitalizaciones (con NaN):", len(df_hospitalizaciones_con))
print("Contagios (con NaN):", len(df_contagios_con))
print("Hospitalizaciones (sin NaN):", len(df_hospitalizaciones_sin))
print("Contagios (sin NaN):", len(df_contagios_sin))

Hospitalizaciones (con NaN): 676087
Contagios (con NaN): 2591793
Hospitalizaciones (sin NaN): 672067
Contagios (sin NaN): 2578099


In [58]:
## Revisión temporal por tipo de causa

# Contamos los registros por año para cada tipo de dataset,
# para ver desde qué año comienzan los registros.

print("Años registrados en CONTAGIOS (con NaN):")
print(df_contagios_con["Anio"].value_counts().sort_index())
print("\nAños registrados en CONTAGIOS (sin NaN):")
print(df_contagios_sin["Anio"].value_counts().sort_index())

print("\nAños registrados en HOSPITALIZACIONES (con NaN):")
print(df_hospitalizaciones_con["Anio"].value_counts().sort_index())
print("\nAños registrados en HOSPITALIZACIONES (sin NaN):")
print(df_hospitalizaciones_sin["Anio"].value_counts().sort_index())


Años registrados en CONTAGIOS (con NaN):
Anio
2014    153524
2015    161420
2016    161728
2017    162778
2018    163548
2019    168098
2020    245974
2021    283500
2022    284715
2023    286434
2024    289089
2025    230985
Name: count, dtype: int64

Años registrados en CONTAGIOS (sin NaN):
Anio
2014    153524
2015    161112
2016    161364
2017    162148
2018    162820
2019    167362
2020    245036
2021    282600
2022    283770
2023    285444
2024    286605
2025    226314
Name: count, dtype: int64

Años registrados en HOSPITALIZACIONES (con NaN):
Anio
2014    21932
2015    23060
2016    23104
2017    23254
2018    23364
2019    24424
2020    78708
2021    94500
2022    94905
2023    95478
2024    96363
2025    76995
Name: count, dtype: int64

Años registrados en HOSPITALIZACIONES (sin NaN):
Anio
2014    21932
2015    23016
2016    23052
2017    23164
2018    23260
2019    24316
2020    78416
2021    94200
2022    94590
2023    95148
2024    95535
2025    75438
Name: count, dtype: int

## Observaciones sobre los tipos de causas

Se identificaron dos grandes grupos dentro de las causas respiratorias:

- **Contagios:** causas como *influenza, covid-19, bronquitis, virus sincicial, adenovirus*, entre otras.
- **Hospitalizaciones:** causas que mencionan explícitamente *“hospitalizaciones”* o *“hospitalización”*.

Los resultados muestran que:

- Los **contagios** comienzan a registrarse desde **2019**, lo que coincide con los primeros brotes de COVID-19.
- Las **hospitalizaciones** aparecen en los datos a partir de **2020**, reflejando el inicio del aumento de casos graves durante la pandemia.

Esta separación permitirá realizar análisis comparativos entre los **niveles de atención ambulatoria (contagios)** 
y los **casos más severos (hospitalizaciones)** a lo largo del tiempo.


## Análisis exploratorio de datos (EDA)

En esta etapa se realizará un análisis de los datos para poder responder las **preguntas de investigación secundarias y la pregunta principal** de nuestro proyecto.

---

### **1. Influencia del tipo de patología en el nivel de urgencia**

**Objetivo:** Analizar cómo el tipo de patología respiratoria se relaciona con la clasificación del nivel de urgencia (leve, moderado, grave).

In [40]:
# cosas que debemos hacer para poder responder esta pregunta
# 1. Seleccionar las columnas relevantes: `Causa` y `TipoUrgencia`.
#relacion_huc = df_hospitalizaciones[["Causa","TipoUrgencia","NivelComplejidad","Anio"]]
#relacion_h.drop_duplicates(subset=["Causa"],inplace=True)
#relacion_huc


,Causa,TipoUrgencia,NivelComplejidad,Anio
1,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,Urgencia Hospitalaria (UEH),Baja Complejidad,2016
2,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,Urgencia Ambulatoria (SUR),Baja Complejidad,2024
9,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,Urgencia ambulatoria,Baja Complejidad,2015
21,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,Urgencia Ambulatoria (SUR),Baja Complejidad,2018
60,HOSPITALIZACIONES POR CAUSAS SISTEMA RESPIRATORIO,Urgencia Hospitalaria (UEH),Baja Complejidad,2017
...,...,...,...,...
3267781,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",Urgencia Ambulatoria (SUR),Baja Complejidad,2022
3267796,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",Urgencia Ambulatoria (SUR),Baja Complejidad,2022
3267828,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",Urgencia Hospitalaria (UEH),Alta Complejidad,2025
3267871,"HOSPITALIZACIONES COVID-19, VIRUS IDENTIFICADO...",Urgencia ambulatoria (SAR),Baja Complejidad,2020


In [41]:
#relacion_cuc = df_contagios[["Causa","TipoUrgencia","NivelComplejidad","Anio"]]
#relacion_cu.drop_duplicates(subset="Causa",inplace=True)
#relacion_cuc

,Causa,TipoUrgencia,NivelComplejidad,Anio
0,IRA Alta (J00-J06),Urgencia ambulatoria (SAR),Baja Complejidad,2015
3,IRA Alta (J00-J06),Urgencia ambulatoria (SAR),Baja Complejidad,2025
8,IRA Alta (J00-J06),Urgencia Ambulatoria (SAPU),Baja Complejidad,2020
12,IRA Alta (J00-J06),Urgencia Hospitalaria (UEH),Baja Complejidad,2016
13,IRA Alta (J00-J06),Urgencia Hospitalaria (UEH),Baja Complejidad,2018
...,...,...,...,...
3267803,"Covid-19, Virus identificado U07.1",Urgencia ambulatoria (SAR),Baja Complejidad,2022
3267827,"Covid-19, Virus identificado U07.1",Urgencia Ambulatoria (SAPU),Baja Complejidad,2021
3267834,"Covid-19, Virus identificado U07.1",Urgencia Ambulatoria (SAPU),Baja Complejidad,2022
3267835,"Covid-19, Virus identificado U07.1",Urgencia Ambulatoria (SAPU),Baja Complejidad,2021


In [42]:
# Agrupar los datos por tipo de patología y nivel de urgencia.
#SAR = data_limpio[data_limpio["TipoUrgencia"]=="Urgencia ambulatoria (SAR)"]
#UEH = data_limpio[data_limpio["TipoUrgencia"]=="Urgencia Hospitalaria (UEH)"]
#SUR = data_limpio[data_limpio["TipoUrgencia"]=="Urgencia Ambulatoria (SUR)"]
#SAPU = data_limpio[data_limpio["TipoUrgencia"]=="Urgencia Ambulatoria (SAPU)"]

recordatorio: crear df causa(contagio o hospitalizacion)-nivel complejidad-año

In [43]:
# contar cuántas atenciones corresponden a cada combinación (leve, moderado, grave)

In [44]:
# Calcular porcentajes o proporciones dentro de cada patología

In [45]:
# Visualizar los resultados en un gráfico de barras o dispersión.

In [46]:
# con esto interpretamos los datos y respondemos la primera pregunta secundaria

### **2. Incidencia geográfica de enfermedades respiratorias**

**Objetivo:** Identificar si existen regiones o comunas con una mayor incidencia de enfermedades respiratorias.


In [47]:
# cosas que debemos hacer para poder resolver esta pregunta
# Seleccionar las columnas `RegionGlosa`, `ComunaGlosa`, `NumTotal`.  
# Agrupar los datos por región y calculamos el total de consultas respiratorias por año.  
# Calcular la tasa de incidencia relativa leí q es algo q se hace con respecto a poblaciones o cosas así.  
# Generar un mapa o gráfico de con la distribución regional (de esos que le gustan al profe los geo no se qué)
# con esto interpretamos los datos y respondemos la segunda pregunta secundaria  

###  **3. Evolución temporal de las patologías respiratorias**

**Objetivo:** Analizar cómo han cambiado las patologías respiratorias a lo largo del tiempo, especialmente antes y después del COVID-19.

In [48]:
# cosas que debemos hacer para poder resolver esta pregunta
# Seleccionar las columnas `Anio`, `SemanaEstadistica`, `Causa`, `NumTotal`.  
# Calculamos el número total de consultas por año y por tipo de patología.  
# Generar una nueva columna “Periodo” (Pre-COVID / Post-COVID).  
# Graficamos las tendencias temporales antes y después del 2020 
#  Comparamos los valores entre ambos periodos, aqui podemos usar cosas de estadisticas, media, moda mediana, desviacion estandar, etc para comparar 
# con esto interpretamos los datos y respondemos la tercera pregunta secundaria  

### **4. Relación entre grupos etarios y prevalencia**

**Objetivo:** Explorar qué patologías afectan con mayor frecuencia a cada grupo etario.


In [49]:
# cosas que debemos hacer para poder resolver esta pregunta
# Seleccionar las columnas con distribución por edad:  `NumMenor1Anio`, `Num1a4Anios`, `Num5a14Anios`, `Num15a64Anios`, `Num65Mas`.  
# Sumar los valores por año o por tipo de patología (es una idea ahi el q haga esta pregunta decide bien) 
# Visualizar la distribución por grupo etario y tipo de causa (gráfico de barras o apilado el profe es más fan del apilado).  
# con esto interpretamos los datos y respondemos la cuarta pregunta secundaria  

## **5. Integración de resultados y pregunta principal**

**Objetivo:** Responder a la pregunta central del proyecto:  
> **¿Se ha observado un aumento de las enfermedades respiratorias que motivan consultas en urgencias tras la pandemia, en comparación con el periodo previo al COVID-19?**

In [50]:
# aquí me cuesta pensar como podríamos integrar lo que obtengamos de cada pregunta
# asi que ayudenme 